# Подключение библиотек

In [ ]:
!pip install tqdm lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import re
import math
import json
import asyncio
import requests
from tqdm.auto import tqdm
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Ссылка на главную страницу сайта

In [ ]:
ROOT_URL = 'https://eda.ru/'

# Парсер основной страницы

In [ ]:
def parse_categories(url) -> dict:
    """
    Производит парсинг категорий и подкатегорий с основной страницы
    """

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    categories = {}
    for category_block in soup.find_all('div', {'class': 'emotion-mnijqh'}):
        category_name = category_block.h3.find(string=True, recursive=False)
        category_link = urljoin(url, category_block.find('a').get('href'))

        subcategories = {}
        for subcategory_block in category_block.findAll('div', {'class': 'emotion-8asrz1'}):
            subcategory_name = subcategory_block.span.find(string=True, recursive=False)
            subcategory_link = urljoin(url, subcategory_block.find('a').get('href'))

            subcategories[subcategory_name] = subcategory_link
        
        categories[category_name] = {
            'lnk': category_link,
            'subcategories': subcategories
        }

    return categories

# Парсер списка рецептов из категории

In [ ]:
def add_recipes_from_page(url_recipes_page, recipes_roll: list, root_url=ROOT_URL):
    """
    Добавляет рецепты в список рецептов
    Принимает адрес страницы с рецептами
    """
    response = requests.get(url_recipes_page)
    soup = BeautifulSoup(response.text, 'lxml')
    
    for recipe_block in soup.find_all('div', {'class': 'emotion-m0u77r'}):
        recipe_image = recipe_block.find('img', {'class': 'emotion-0'}).get('src')
        recipe_image = None if recipe_image.startswith('data:') else recipe_image
        recipe_name = getattr(recipe_block.find('span', {'class': 'emotion-1pdj9vu'}), 'text', None)
        recipe_link = urljoin(root_url, recipe_block.find('a', {'class': 'emotion-18hxz5k'}).get('href'))
        
        recipes_roll.append({
            'name': recipe_name,
            'img': recipe_image,
            'lnk': recipe_link,
        })

In [ ]:
def parse_recipes_from_category(category_url, category_name, start_page=1, stop_page=None) -> list:
    """
    Парсер рецептов из конкретной категории
    Пример категории: Торты - https://eda.ru/recepty/torty
    """
    recipes_roll = []

    if stop_page is None:
        # Кол-во рецептов на одной странице
        amount_recipes_on_page = 14
        # Определение общего количества рецептов данной категории
        total_recipes_amount = int(re.search(r'\d+',
                                             BeautifulSoup(requests.get(category_url).text, 'lxml').title.text).group(0))
        
        stop_page = math.ceil(total_recipes_amount / amount_recipes_on_page)
    
    for page_number in tqdm(range(start_page, stop_page + 1), desc=category_name, leave=False, colour='blue'):
        url_recipes_page = category_url + f'?page={page_number}'
        add_recipes_from_page(url_recipes_page, recipes_roll)

    return recipes_roll

# Парсер конкретного рецепта

In [ ]:
def parse_recipe(html: str,
                 url_recipe: str,
                 image=None) -> dict:
        """
        Парсит конкретный рецепт по абсолютной ссылке
        """
        soup = BeautifulSoup(html, 'lxml')

        recipe = {}

        recipe['Название'] = getattr(soup.find('h1', {'class': 'emotion-gl52ge'}), 'text', None)
        recipe['Ссылка на рецепт'] = url_recipe
        recipe['Изображение'] = image

        recipe['Время приготовления'] = getattr(soup.find('div', {'class': 'emotion-my9yfq'}), 'text', None)
        recipe['Количество порций'] = getattr(soup.find('span', {'itemprop': "recipeYield"}), 'text', None)

        recipe['Энергетическая ценность на порцию'] = {
            'калорийность': getattr(soup.find('span', {'itemprop': "calories"}), 'text', None),
            'белки': getattr(soup.find('span', {'itemprop': "proteinContent"}), 'text', None),
            'жиры': getattr(soup.find('span', {'itemprop': "fatContent"}), 'text', None),
            'углеводы': getattr(soup.find('span', {'itemprop': "carbohydrateContent"}), 'text', None),
        }

        recipe['Ингредиенты'] = {
            getattr(ingredient.find('span', {'itemprop': 'recipeIngredient'}), 'text', None):
            getattr(ingredient.find('span', {'class': 'emotion-bsdd3p'}), 'text', None)
            for ingredient in soup.find_all('div', {'class': 'emotion-7yevpr'})
        }

        recipe['Описание'] = getattr(soup.find('span', {'class': 'emotion-aiknw3'}), 'text', None)

        recipe['Инструкция приготовления'] = [step.text for step in soup.find_all('span', {'class': 'emotion-wdt5in'})]

        recipe['Совет к рецепту'] = getattr(soup.find('div', {'class': 'emotion-1mtrnmn'}), 'text', None)

        return recipe

In [ ]:
def sync_parse_recipe(url: str, image=None):
    response = requests.get(url)
    return parse_recipe(response.text, url, image)

In [ ]:
async def async_parse_recipe(session, url: str, recipes: dict, image=None):
    async with session.get(url) as resp:
        assert resp.status == 200
        resp_text = await resp.text()
        recipes[url] = parse_recipe(resp_text, url, image)

In [ ]:
async def async_parse_recipes(recipes: list, write_to: dict):
    async with aiohttp.ClientSession() as session:
        
        tasks = []
        for recipe in recipes:
            recipe_name, recipe_link, recipe_image = recipe['name'], recipe['lnk'], recipe['img']

            if recipe_link not in write_to:
                task = asyncio.create_task(async_parse_recipe(session, recipe_link, write_to, recipe_image))
                tasks.append(task)

        await asyncio.gather(*tasks)

# Функции для сохранений и загрузки

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
PATH = '/content/drive/MyDrive/Colab Notebooks/ChefAI/'

In [ ]:
os.listdir(PATH)

['recipes_from_categories.json',
 '.ipynb_checkpoints',
 'all_categories.json',
 'Заготовки.json',
 'all_recipes.json',
 'Filnal-parse_eda.ru.ipynb']

In [ ]:
def dump(obj, filename='dump', path=PATH):
    with open(os.path.join(path, f'{filename}.json'), 'w', encoding='utf-8') as json_file:
        json.dump(obj, json_file, indent=2, ensure_ascii=False)

In [ ]:
def load(filename: str, path=PATH):
    return json.loads(open(os.path.join(path, f'{filename}.json'), 'r', encoding='utf-8').read())

In [ ]:
def exist(filename: str, path=PATH):
    return f'{filename}.json' in os.listdir(path)

# Парсинг категорий

In [ ]:
all_categories_filename = 'all_categories'
if exist(all_categories_filename):
    categories = load(all_categories_filename)
else:
    categories = parse_categories(ROOT_URL)
    dump(categories, all_categories_filename)

# Парсинг списка рецептов для каждой категории и подкатегорий

In [ ]:
recipes_from_categories_filename = 'recipes_from_categories'
if exist(recipes_from_categories_filename):
    recipes_from_categories = load(recipes_from_categories_filename)
else:
    recipes_from_categories = {}

    while len(recipes_from_categories) < len(categories):
        for category_name, category_data in tqdm(categories.items(), desc='Парсинг категорий'):
            if category_name in recipes_from_categories:
                continue
            
            if exist(category_name):
                recipes_from_categories[category_name] = load(category_name)
                continue

            category_link, subcategories = category_data['lnk'], category_data['subcategories']
            
            # парсим основную категорию
            try:
                category = {category_name: parse_recipes_from_category(category_link, category_name)}
            except:
                break

            # парсим все подкатегории
            while len(category) < len(subcategories):
                for subcategory_name, subcategory_link in tqdm(subcategories.items(), desc=category_name, leave=False):
                    if subcategory_name in category:
                        continue
                    try:
                        category[subcategory_name] = parse_recipes_from_category(subcategory_link, subcategory_name)
                    except:
                        break
            
            # Запись категории в словарь со всеми категориями и рецептами
            recipes_from_categories[category_name] = category

            # сохраняем категорию
            dump(category, category_name)

    # Сохраняем финальный результат (Списоки поверхностной информации о рецептах для всех категорий и подкатегорий)
    dump(recipes_from_categories, recipes_from_categories_filename)

# Подготовка к парсингу рецептов

In [ ]:
total_recipes = len({recipe['lnk']
                     for subcategories in recipes_from_categories.values()
                     for subcategory in subcategories.values()
                     for recipe in subcategory})

In [ ]:
timeout = 30
retry = None

# Синхронный парсинг всех рецептов

In [ ]:
all_recipes_filename = 'all_recipes'
all_recipes_tmp_filename = 'tmp'

if exist(all_recipes_filename):
    all_recipes = load(all_recipes_filename)
elif exist(all_recipes_tmp_filename):
    all_recipes = load(all_recipes_tmp_filename)
else:
    all_recipes = {}

In [ ]:
print(len(all_recipes), total_recipes)

47241 47241


In [ ]:
amount = 0
while len(all_recipes) < total_recipes:
    try:
        for category_name, subcategories in tqdm(recipes_from_categories.items(),
                                                 desc='Парсинг рецептов'):
            for subcategory_name, recipes_roll in tqdm(subcategories.items(),
                                                       desc=category_name,
                                                       leave=False):
                for recipe in tqdm(recipes_roll,
                                   desc=subcategory_name,
                                   leave=False):
                    if recipe['lnk'] in all_recipes:
                        continue

                    all_recipes[recipe['lnk']] = sync_parse_recipe(recipe['lnk'], recipe['img'])

    except Exception as e:
        print(e)
        time.sleep(timeout)
        amount += 1

    except KeyboardInterrupt as e:
        break
        
    finally:
        dump(all_recipes, all_recipes_tmp_filename)
        if retry and amount >= retry:
            break

dump(all_recipes, all_recipes_filename)

# Асинхронный парсинг всех рецептов

In [ ]:
!pip install aiohttp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio

In [ ]:
def take_by(roll: list, amount: int = 1):
    for i in range(0, len(roll), amount):
        yield roll[i:i + amount]

In [ ]:
number_requests = 3

In [ ]:
amount = 0
while len(all_recipes) < total_recipes:
    try:
        for category_name, subcategories in tqdm(recipes_from_categories.items(), desc='Парсинг рецептов'):
            for subcategory_name, recipes_roll in tqdm(subcategories.items(), desc=category_name, leave=False):
                with tqdm(total=len(recipes_roll), desc=subcategory_name, leave=False) as pbar:
                    for recipes in take_by(recipes_roll, number_requests):
                        asyncio.run(async_parse_recipes(recipes, all_recipes))
                        pbar.update(len(recipes))
    except Exception as e:
        time.sleep(timeout)
        amount += 1
        
    except KeyboardInterrupt as e:
        break

    finally:
        dump(all_recipes, all_recipes_tmp_filename)
        if retry and amount >= retry:
            break
dump(all_recipes, all_recipes_filename) 

# Повторный парсинг проблемных рецептов

In [ ]:
# поиск проблемных рецептов
problems_recipes = {url_recipe:recipe['Изображение'] for url_recipe, recipe in all_recipes.items()
                    if recipe['Название'] is None}
print(len(problems_recipes))

0


In [ ]:
problems_recipes

{}

In [ ]:
for recipe_url, image_url in problems_recipes.items():
    all_recipes[recipe_url] = sync_parse_recipe(recipe_url, image_url)

In [ ]:
del all_recipes['https://eda.ru/recepty/salaty/zelenyy-miks-salat-s-file-greypfruta-kurinym-file-na-grile-sousom-iz-domashnego-yogurta-s-travami-na-sloenom-teste-151525']

In [ ]:
dump(all_recipes, all_recipes_filename) 

In [ ]:
recipes = {
    'https://eda.ru/recepty/zakuski/karelskie-pirozhki-s-kartoshkoj--kalitki--42966':
    'https://eda.ru/img/eda/c180x180/s1.eda.ru/StaticContent/Photos/160812092111/160819101118/p_O.jpg',

    'https://eda.ru/recepty/osnovnye-blyuda/sibas-s-cherri-27121':
    'https://eda.ru/img/eda/c180x180/s1.eda.ru/StaticContent/Photos/110809195434/120214140151/p_O.jpg',

    'https://eda.ru/recepty/osnovnye-blyuda/kurinaja-grudka-zapechennaja-s-rozmarinom-timjanom-i-cedroj-citrusovih-53526':
    'https://eda.ru/img/eda/c180x180/s1.eda.ru/StaticContent/Photos/120213180404/151109015855/p_O.jpg',

    'https://eda.ru/recepty/salaty/zelenyy-miks-salat-s-file-greypfruta-kurinym-file-na-grile-sousom-iz-domashnego-yogurta-s-travami-na-sloenom-teste-151525':
    None,

    'https://eda.ru/recepty/vypechka-deserty/fruktovij-salat-s-apelsinom-marakujej-23145':
    'https://eda.ru/img/eda/c180x180/s1.eda.ru/StaticContent/Photos/120214124236/120214124811/p_O.jpg',

    'https://eda.ru/recepty/napitki/kofejno-slivochnij-koktejl-s-sambukoj-22727':
    'https://eda.ru/img/eda/c180x180/s1.eda.ru/StaticContent/Photos/120131111725/120214123727/p_O.jpg',

    'https://eda.ru/recepty/vypechka-deserty/blini-s-cukini-morkovju-24368':
    'https://eda.ru/img/eda/c180x180/s1.eda.ru/StaticContent/Photos/120214130615/120214131015/p_O.jpg',

    'https://eda.ru/recepty/vypechka-deserty/oladi-iz-kurinogo-mjasa-gerkulesa-28359':
    'https://eda.ru/img/eda/c180x180/s1.eda.ru/StaticContent/Photos/120214133254/120214141842/p_O.jpg',

    'https://eda.ru/recepty/vypechka-deserty/bulochki-s-rozmarinom-i-sirnoj-nachinkoj-53723':
    None,

    'https://eda.ru/recepty/vypechka-deserty/bananovye-mini-bulochki-s-kuragoy-92244':
    'https://eda.ru/img/eda/c180x180/s1.eda.ru/StaticContent/Photos/120214125956/170217192701/p_O.jpg',

    'https://eda.ru/recepty/vypechka-deserty/angliyskie-sdobnye-bulochki-k-chayu-pri-diabete-68938':
    'https://eda.ru/img/eda/c180x180/s1.eda.ru/StaticContent/Photos/121120172110/160416201458/p_O.jpg'
}

parsed_recipes = {}
for recipe_url, image_url in recipes.items():
    parsed_recipes[recipe_url] = sync_parse_recipe(recipe_url, image_url)